## This script generates a zoomable map for a single year of the Lymantria Trap data

#### Import the needed libraries

In [22]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from mapbox import Geocoder

geocoder = Geocoder(access_token=".mapbox_tokenN")

#### The following function generates an appropriate map window and map zoom absed on thenGPS coordinates in the data

In [23]:
def zoom_center(lons: tuple = None, lats: tuple = None, lonlats: tuple = None,
                format: str = 'lonlat', projection: str = 'mercator',
                width_to_height: float = 2.0) -> (float, dict):
    """Finds optimal zoom and centering for a plotly mapbox.
    Must be passed (lons & lats) or lonlats.
    Temporary solution awaiting official implementation, see:
    https://github.com/plotly/plotly.js/issues/3434

    Parameters
    --------
    lons: tuple, optional, longitude component of each location
    lats: tuple, optional, latitude component of each location
    lonlats: tuple, optional, gps locations
    format: str, specifying the order of longitud and latitude dimensions,
        expected values: 'lonlat' or 'latlon', only used if passed lonlats
    projection: str, only accepting 'mercator' at the moment,
        raises `NotImplementedError` if other is passed
    width_to_height: float, expected ratio of final graph's with to height,
        used to select the constrained axis.

    Returns
    --------
    zoom: float, from 1 to 20
    center: dict, gps position with 'lon' and 'lat' keys

#    >>> print(zoom_center((-109.031387, -103.385460),
    ...     (25.587101, 31.784620)))
    (5.75, {'lon': -106.208423, 'lat': 28.685861})
    """
    if lons is None and lats is None:
        if isinstance(lonlats, tuple):
            lons, lats = zip(*lonlats)
        else:
            raise ValueError(
                'Must pass lons & lats or lonlats'
            )

    maxlon, minlon = max(lons), min(lons)
    maxlat, minlat = max(lats), min(lats)
    center = {
        'lon': round((maxlon + minlon) / 2, 6),
        'lat': round((maxlat + minlat) / 2, 6)
    }

    # longitudinal range by zoom level (20 to 1)
    # in degrees, if centered at equator
    lon_zoom_range = np.array([
        0.0007, 0.0014, 0.003, 0.006, 0.012, 0.024, 0.048, 0.096,
        0.192, 0.3712, 0.768, 1.536, 3.072, 6.144, 11.8784, 23.7568,
        47.5136, 98.304, 190.0544, 360.0
    ])

    if projection == 'mercator':
        margin = 1.2
        height = (maxlat - minlat) * margin * width_to_height
        width = (maxlon - minlon) * margin
        lon_zoom = np.interp(width, lon_zoom_range, range(20, 0, -1))
        lat_zoom = np.interp(height, lon_zoom_range, range(20, 0, -1))
        zoom = round(min(lon_zoom, lat_zoom), 2)
    else:
        raise NotImplementedError(
            f'{projection} projection is not implemented'
        )

    return zoom, center


#### Read in the data

In [24]:
df = pd.read_csv('/Users/Douglas/Documents/GitHub/Lymantria/data/combined_csv.csv')
df

,POINT_X,POINT_Y,count,year
0,-78.349984,35.395156,0,1988
1,-81.043286,35.265153,0,1988
2,-81.019992,35.288449,0,1988
3,-81.059996,35.291753,0,1988
4,-81.079981,35.288451,0,1988
...,...,...,...,...
3021792,-83.376008,40.082736,0,2019
3021793,-83.407256,40.086289,0,2019
3021794,-83.452638,40.078905,0,2019
3021795,-83.142491,40.089619,0,2019


#### The followng manipulate the data in some useful ways

In [25]:
# Select the year of data we want to map
df = df[df['year'] == 2016]

# define the range of lat/long points to set the limits of the map
min_x = df['POINT_X'].min()
max_x = df['POINT_X'].max()
min_y = df['POINT_Y'].min()
max_y = df['POINT_Y'].max()

# define the format of the text that appears when you hover over a data point on the pa
df['text'] = '<br>Moths Trapped ' + (df['count']).astype(str)

# a list that contains the lower, upper limits for 5 bins of trap counts
limits = [(0, 1), (1, 20), (20, 50), (50, 200), (200, 3000)]

# a list that contains the colors that correspond to each bin size of  trap counts
colors = ["lightseagreen", "yellow", "orange", "crimson", "red"]
scale = 5000


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Call the function that returns the center and the zoom leve of the map

In [26]:
zoom, center = zoom_center(
    lons=[min_x, max_x],
    lats=[min_y - 3, max_y + 4])

#### Generate the map itself

In [27]:
fig = go.Figure()

# This loop iterates through each bin size, and assigns a size and color to the markers on the map
for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[(df["count"] >= lim[0]) & (df["count"] < lim[1])]
    fig.add_trace(go.Scattermapbox(
       lon=df_sub['POINT_X'],
        lat=df_sub['POINT_Y'],
        text=df_sub['text'],
        marker=dict(
            size=(df_sub['count'] / 1.5 + 10),
            color=colors[i],
            sizemode='area',
        ),
        name='{0} - {1}'.format(lim[0], lim[1])))

#### Just some figure style updates, then write to an html file

In [30]:
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox={'center': center, 'zoom': zoom},
    title_text='2016 Lymantria Trap Data',
    showlegend=True,
)
fig.write_html("2016.html")